```
bryan@ichabod mac_bucket % for file in *_dst.json; do
  databricks fs cp "$file" dbfs:/Volumes/geodata/petra/dst/
done
```

In [ ]:
df = spark.read.format('json').load("/Volumes/geodata/petra/dst/")

display(df)

In [ ]:
from common.transforms import string_to_iso_date, generate_hash
import pyspark.sql.functions as F

df_flat = df.select(
    F.col("repo_id"),
    F.col("uwi.uwi").alias("uwi"),
    F.col("uwi.wsn").alias("wsn"),
    F.col("fmtest.base").alias("base"),
    F.col("fmtest.top").alias("top"),
    F.col("fmtest.date").alias("date"),
    F.col("fmtest.fmname").alias("formation"),
    F.col("fmtest.numrecov").alias("num_recovered"),
    F.col("fmtest.recov").alias("recov"),
    F.col("fmtest.remark").alias("remark"),
    F.col("fmtest.chgdate").alias("app_row_changed"),
)

date_columns = [
    "date",
    "app_row_changed"
]

df_dst = df_flat


# enforce timestamp for dates
for col_name in date_columns:
    df_dst = string_to_iso_date(df_dst, col_name, col_name)

# add id hash
id_columns = ["repo_id", "uwi"]
df_dst = generate_hash(df_dst, "id", "dst", *id_columns)


display(df_dst)